In [9]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd 
import datetime
import re

In [10]:
KEYWORDS = ['AMD', 'Intel', 'IBM', 'Oracle', 'Google', 'Apple', 'Microsoft']

In [11]:
def date_change(string):
    date = r'(\d{4}-\d{2}-\d{2}) \d{2}:\d{2}:\d{2}.\d{6}сегодня в (\d{2}:\d{2})'
    replace = r'\1 \2'
    return re.sub(date, replace, string)

In [12]:
def time_test(string):
    time = r'(.+) в \d{2}:\d{2}'
    replace = r'\1'
    return re.sub(time, replace, string)

In [13]:
def clean_html(string):
    tags = r'\\r|\\n'
    replace = r' '
    return re.sub(tags, replace, string)

### распарсим одну страницу

In [14]:
time_now = str(datetime.datetime.now())
parsing_dict_list = []
# список из превью по странице
post_preview = soup.find_all(class_='post post_preview')

for post in post_preview:
    # вытащить превью
    link_preview = post.find_all(class_='post__title_link')
    if link_preview:
        # выташить линки и время
        url = link_preview[0].get('href')
        time_preview = post.find_all(class_='post__time')
    # еще один вариант нужных нам ссылок
    else:
        link_preview = post.find_all(class_='preview-data__title-link')
        time_preview = post.find_all(class_='preview-data__time-published')
    url = link_preview[0].get('href')
    time = time_preview[0].get_text()
    header = link_preview[0].get_text()
    text = post.get_text()
    date_news = date_change(time_now+time)
    for key in KEYWORDS:
        if text.find(key) > 0:
            record = {'Tag':key, 'Date':date_news, 'URL':url, 'Header':header}
            parsing_dict_list.append(record)

pd.DataFrame(parsing_dict_list)

NameError: name 'soup' is not defined

### а теперь самое интересное - вытащим со всех страниц новостей только сегодняшние новости с требуемыми компаниями

In [15]:
time_now = str(datetime.datetime.now())
parsing_dict_list = []
i = 1
timestamp = 'сегодня'

while timestamp == 'сегодня':
    
    # самое забавное, что я был немного не в форме во время лекции, здоровье подвело, и момент с передачей параметров 
    # я как-то вообще пролюбил и только когда уже пересматривал потом запись, врубился в технику посылания запросов с ними
    # так что переключать страницы можно и покрасивее возможно, но этот код я написал еще до этого, просто придумав свое
    # немного костыльное решение с формированием URL по ходу дела и оно сработало!
    # ну раз работает, я решил не переделывать этот кусок в мемориальных целях
    
    url = 'https://habr.com/ru/all/page' + str(i)
    req_page = req.get(url)
    soup = BeautifulSoup(req_page.text, 'html.parser')
    post_preview = soup.find_all(class_='post post_preview')
    for post in post_preview:
        link_preview = post.find_all(class_='post__title_link')
        if link_preview: 
        # нужный нам элемент выдирается разными способами в зависимости от того статья это или часть корпоративного блога
            url = link_preview[0].get('href')
        # тег div я тоже явно не прописывал, но оно все лежит и так там, где описано
            time_preview = post.find_all(class_='post__time')
        else:
            link_preview = post.find_all(class_='preview-data__title-link')
            time_preview = post.find_all(class_='preview-data__time-published')
        url = link_preview[0].get('href')
        time = time_preview[0].get_text()
        header = link_preview[0].get_text()
        text = post.get_text()
        date_news = date_change(time_now+time)
        timestamp = time_test(time)
        if timestamp == 'сегодня':
            for key in KEYWORDS:
                if text.find(key) > 0:
                    record = {'Tag':key, 'Date':date_news, 'URL':url, 'Header':header}
                    parsing_dict_list.append(record)
        else:
            break
    i += 1
pd.DataFrame(parsing_dict_list)

,Tag,Date,URL,Header
0,Google,2020-11-20 11:55,https://habr.com/ru/company/vdsina/blog/528990/,"Окей, Гугл, опубликуй свои секретные ключи DKIM"
1,Google,2020-11-20 03:55,https://habr.com/ru/company/selectel/blog/529010/,Ubuntu Web Remix — альтернатива Chrome OS c бр...


### теперь попробуем распарсить текст статьи по каждой ссылке

In [16]:
time_now = str(datetime.datetime.now())
parsing_dict_list = []
i = 1
timestamp = 'сегодня'

while timestamp == 'сегодня':
    url = 'https://habr.com/ru/all/page' + str(i)
    req_page = req.get(url)
    soup = BeautifulSoup(req_page.text, 'html.parser')
    post_preview = soup.find_all(class_='post post_preview')
    for post in post_preview:
        link_preview = post.find_all(class_='post__title_link')
        if link_preview:
            url = link_preview[0].get('href')
            time_preview = post.find_all(class_='post__time')
        else:
            link_preview = post.find_all(class_='preview-data__title-link')
            time_preview = post.find_all(class_='preview-data__time-published')
        url = link_preview[0].get('href')
        time = time_preview[0].get_text()
        header = link_preview[0].get_text()
        date_news = date_change(time_now+time)
        timestamp = time_test(time)       
        # заход на страницу
        req_page_inner = req.get(url)
        soup_inner = BeautifulSoup(req_page_inner.text, 'html.parser')
        full_text = soup_inner.find_all(id="post-content-body")
        html = full_text[0].get_text()
        clean_text = clean_html(html)
        if timestamp == 'сегодня':
            for key in KEYWORDS:
                if clean_text.find(key) > 0:
                    record = {'Tag':key, 'Date':date_news, 'URL':url, 'Full text':clean_text}
                    parsing_dict_list.append(record)
        else:
            break
    i += 1
pd.DataFrame(parsing_dict_list)
# какого-то фига у меня не сработал регексп, полностью очищающий от всяких \n\n хотя 
# в regexp101 он нормально чистит, я проверял

,Tag,Date,URL,Full text
0,Microsoft,2020-11-20 12:27,https://habr.com/ru/post/528914/,"Я проработал в сфере IT около 25 лет, из котор..."
1,Google,2020-11-20 11:55,https://habr.com/ru/company/vdsina/blog/528990/,\n\r\nИнтернет даже в лучшие свои годы был опа...
2,Apple,2020-11-20 11:50,https://habr.com/ru/post/528950/,Кто сегодня не слышал про вебинары – реалии се...
3,Google,2020-11-20 11:26,https://habr.com/ru/post/529004/,"\n\n<< До этого: Восхождение интернета, ч.2: п..."
4,Apple,2020-11-20 11:26,https://habr.com/ru/post/529004/,"\n\n<< До этого: Восхождение интернета, ч.2: п..."
5,Microsoft,2020-11-20 11:26,https://habr.com/ru/post/529004/,"\n\n<< До этого: Восхождение интернета, ч.2: п..."
6,Google,2020-11-20 11:14,https://habr.com/ru/company/jugru/blog/527508/,Soft skills крайне важны для DevOps-специалист...
7,Google,2020-11-20 10:51,https://habr.com/ru/company/mailru/blog/528960/,\n\r\nВсем привет! В нашем совместном с МФТИ у...
8,Intel,2020-11-20 10:50,https://habr.com/ru/company/ua-hosting/blog/52...,"\n\r\nЧто общего между молоком, сливочным масл..."
9,Intel,2020-11-20 10:13,https://habr.com/ru/post/529020/,\n\r\nВ данной публикации будет описан немного...


### Теперь поколупаем скрытые API

In [133]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [189]:
url = 'https://www.avast.com/hackcheck/'
data = {"emailAddresses":[EMAIL[0]]}

headers = {
'Connection': 'keep-alive',
'Content-Type': 'application/json;charset=UTF-8',
'Host': 'identityprotection.avast.com',
'Origin': 'https://www.avast.com',
'Referer': 'https://www.avast.com/',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Site': 'same-site',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

response = req.post(url, data, headers)

In [190]:
response

<Response [400]>

В общем я извращался над запросом ВСЕМИ способами просто - пихал нагрузку в json и не пихал, копировал все что можно из заголовка, ну как бы больше вариантов я, увы, не знаю, ибо в сайтостроении я шарю исключительно в самом самой самой общей теории и чего еще ему от меня надо я понять не в состоянии. Куков он никаких не ждет и не отправляет, я смотрел в инспекторе,но даже если бы и ждал - я не нашел ни одного адекватного туториала, как их из тормозиллы выколупать и прилепить к запросу. Все ответы на вопросы во всяких чатах обычно подразумевают уже нехилое знание спрашивающим веба. Так что, увы, вебмастер из меня нулевой абсолютно и я даже реально вообще не в состоянии осознать - чего он в приниципе еще может хотеть от меня??